# Visualizing Genetic Circuits: Creating Diagrams with SBOL Visual

In this notebook, we will explore how to load a transcriptional unit from an RDF file, extract and order its components, and visualize the genetic circuit using two different libraries: `dnaplotlib` and `parasbolv`. We will also customize the visual output by defining specific styles for each type of genetic part, such as promoters, RBS, CDS, and terminators.


In [1]:
# Setup
# Install the necessary libraries
# pip install sbol3 excel2sbol dnaplotlib parasbolv

import sbol3
import dnaplotlib as dpl
import matplotlib.pyplot as plt
import parasbolv as psv
from collections import namedtuple
from matplotlib import gridspec
gs = gridspec.GridSpec(1, 1)

### Importing Libraries

- **sbol3**: The SBOL3 library allows us to handle and manipulate SBOL documents, which store genetic circuit information.
- **matplotlib**: A widely used library for creating visualizations in Python.
- **dnaplotlib**: A specialized library for visualizing DNA sequences and genetic circuits.
- **parasbolv**: A light-weight library for visualizing DNA sequences and genetic circuits.

In [2]:
# Load the SBOL document from the RDF file
doc = sbol3.Document()
doc.read('transcriptional_unit.rdf')

### Retrieving Ordered Components from a Transcriptional Unit

This function, `get_ordered_components`, extracts the components in the correct sequential order from a transcriptional unit based on the constraints defined in the SBOL3 document. It does so by identifying the `SubComponent` instances within the transcriptional unit and resolving them to their corresponding `Component` objects using the `instance_of` property. The function ensures the correct order of components by traversing the "meets" relationships, which specify how one component follows another in the sequence.


In [3]:
# Define the function to extract ordered components with orientation
def get_ordered_components(doc, transcriptional_unit_id):

    # Retrieve the transcriptional unit component
    transcriptional_unit = doc.find(transcriptional_unit_id)
    
    # Create a dictionary to hold the order of components
    order_map = {}
    
    # Populate the order_map based on the constraints
    for constraint in transcriptional_unit.constraints:
        if constraint.restriction == sbol3.SBOL_MEETS:  # Check if the constraint is a 'meets' relationship
            subject = constraint.subject.lookup()
            obj = constraint.object.lookup()
            order_map[subject.display_id] = obj.display_id
    
    # Use the order_map to determine the order of components
    ordered_components = []
    
    # Find the first component (one that is not in the object of any constraint)
    first_subcomponent = None
    for feature in transcriptional_unit.features:
        if isinstance(feature, sbol3.SubComponent) and feature.display_id not in order_map.values():
            first_subcomponent = feature
            break
    
    # Traverse the map to get the ordered list of components
    current_subcomponent = first_subcomponent
    while current_subcomponent:
        # Get the associated component from the instance_of property
        component = current_subcomponent.instance_of.lookup()
        ordered_components.append(component)
        next_subcomponent_id = order_map.get(current_subcomponent.display_id)
        current_subcomponent = transcriptional_unit.find(next_subcomponent_id) if next_subcomponent_id else None
    
    return ordered_components

### Extracting and Ordering Components

The transcriptional unit contains multiple genetic parts, such as promoters and CDS, which need to be ordered correctly to accurately represent the genetic circuit. We use a custom function to extract these components and determine their correct order based on the constraints defined in the SBOL document.

In [4]:
ordered_components = get_ordered_components(doc, 'https://github.com/SynBioDex/SBOL-Notebooks/TranscriptionalUnit1')

# Styling to DNAplotlib Visual Elements

We now create a helper function `get_dnaplotlib_part` that converts each SBOL3 `Component` into a DNAPlotlib Visual Element. Each visual element is a dictionary that has the following properties: 
- part type (`type`)
- orientation (`fwd`)
- styling options (`opts`)

In this section - we'll use the default styling options in dnaplotlib.

We then store these dictionaries in a `list` called `design`. 

In [5]:

# SBOL Type to DNAPlotlib Type Map:
visual_type_map = {
    sbol3.SO_PROMOTER: 'Promoter',
    sbol3.SO_RBS: 'RBS',
    sbol3.SO_CDS: 'CDS',
    sbol3.SO_TERMINATOR: 'Terminator'
}

def get_dnaplotlib_part(component: sbol3.Component):
    return {
        'type': visual_type_map[component.roles[0]],
        'fwd': True
    }

design = [get_dnaplotlib_part(x) for x in ordered_components]

Next, we'll create a helper function `get_plot_from_design` that can convert a dnaplotlib `design` into a `.png` file.

In [6]:
def get_plot_from_design(design, filename):
    fig = plt.figure(figsize=(1,0.3))
    ax_dna = plt.subplot(gs[0])

    # Create the DNAplotlib renderer
    dr = dpl.DNARenderer()
    dr.linewidth = 0.8

    start, end = dr.renderDNA(ax_dna, design, dr.SBOL_part_renderers())
    ax_dna.set_xlim([start, end])
    ax_dna.set_ylim([-10,30.0])
    ax_dna.set_aspect('equal')
    ax_dna.axis('off')

    fig.savefig(f'{filename}.png', dpi=300, transparent=True)
    plt.close('all')

In [7]:
get_plot_from_design(design, 'tu')

### Adding Custom Properties and Styling to DNAplotlib Visual Elements

In this section, we define a property map, `dpl_property_map`, that specifies custom styling options for each type of genetic part (e.g., promoters, RBS, CDS, terminators). These properties include line width, color, and other visual attributes, allowing us to customize the appearance of each part when rendering with DNAplotlib.

We then define a helper function, `get_dnaplotlib_part_props`, which maps each SBOL component to its corresponding DNAplotlib properties based on its role. This function returns a dictionary with the part type, orientation, and styling options.

Using this function, we generate a list of part properties for the ordered components and render the transcriptional unit with DNAplotlib, applying the specified styles. The final figure is saved as a PNG file with the specified custom styling.

In [8]:
dpl_property_map = {
    sbol3.SO_PROMOTER: {'linewidth':0.8, 'color':'red', 'edge_color':(0.00, 0.00, 0.00), 'y_extent':9, 'x_extent':13, 'arrowhead_height':2, 'arrowhead_length':3},
    sbol3.SO_RBS: {'linewidth':0.3, 'color':'black', 'edge_color':(0.00, 0.00, 0.00), 'x_extent':7, 'start_pad':1},
    sbol3.SO_CDS: {'linewidth':0.2, 'color':'green', 'edge_color':(0.00, 0.00, 0.00), 'x_extent':14, 'y_extent':2.2,'arrowhead_height':2,'arrowhead_length':4},
    sbol3.SO_TERMINATOR: {'linewidth':0.5, 'color':'black', 'edge_color':(0.00, 0.00, 0.00), 'x_extent':6, 'y_extent':6}
}

def get_dnaplotlib_part_props(component: sbol3.Component):
    return {
        'type': visual_type_map[component.roles[0]],
        'fwd': True,
        'opts': dpl_property_map[component.roles[0]]
    }

design_with_props = [get_dnaplotlib_part_props(x) for x in ordered_components]
get_plot_from_design(design_with_props, 'tu_color')


# Visualising using the ParaSBOLv library

The ParaSBOLv allows users to use custom glyphs to create SBOL Visual diagrams. In this example, we've copied the default glyphs from the parasbolv library to this folder. You can modify the glyphs to see how your visual elements change using parasbolv.

In [9]:
# Initialize the Parasbolv renderer and load the glyphs library.
renderer = psv.GlyphRenderer('glyphs')

In [12]:
# We are creating a tuple to make it easier to store properties of visual glyphs.
Part = namedtuple('part', ['glyph_type', 'orientation',  'user_parameters', 'style_parameters'])


# SBOL Type to ParaSBOLv Type Map:
visual_type_map = {
    sbol3.SO_PROMOTER: 'Promoter',
    sbol3.SO_RBS: 'RibosomeEntrySite',
    sbol3.SO_CDS: 'CDS',
    sbol3.SO_TERMINATOR: 'Terminator'
}


def get_psv_part(component: sbol3.Component):
    
    return Part(visual_type_map[component.roles[0]],
                'forward',
                None,
                None)

part_list = [get_psv_part(x) for x in ordered_components]

construct = psv.Construct(part_list, renderer)
fig, ax, baseline_start, baseline_end, bounds = construct.draw()
ax.plot([baseline_start[0], baseline_end[0]], [baseline_start[1], baseline_end[1]], color=(0,0,0), linewidth=1.5, zorder=0)

# Save the visualization as an image
plt.savefig('tu_parasbolv.png', dpi=300, transparent=True)

Like DNAplotlib, we can customise the visual properties of the glyphs. In this case, we'll make the CDS green with a blue outline.

In [14]:
psv_property_map = {
    sbol3.SO_CDS: {'cds': {'facecolor': (0,1,0),
                               'edgecolor': (0,0,1), 'linewidth': 2}}
} 

def get_psv_part_props(component: sbol3.Component):
    
    return Part(visual_type_map[component.roles[0]],
                'forward',
                None,
                psv_property_map.get(component.roles[0], None))

part_list = [get_psv_part_props(x) for x in ordered_components]

construct = psv.Construct(part_list, renderer)
fig, ax, baseline_start, baseline_end, bounds = construct.draw()
ax.plot([baseline_start[0], baseline_end[0]], [baseline_start[1], baseline_end[1]], color=(0,0,0), linewidth=1.5, zorder=0)

# Save the visualization as an image
plt.savefig('tu_parasbolv_color.png', dpi=300, transparent=True)

### Conclusion

In this notebook, we demonstrated how to load, analyze, and visualize a transcriptional unit using two powerful tools: DNAplotlib and Parasbolv. We began by loading an SBOL3 transcriptional unit from an RDF file, extracting and ordering its components based on the defined constraints. 

We then explored how to customize the visual representation of the genetic circuit using DNAplotlib, applying specific styles to each part to create a clear and informative diagram.

Finally, we introduced Parasbolv as an alternative visualization tool that adheres closely to the SBOL Visual standard, offering a different aesthetic for representing genetic circuits. Both visualization methods provide valuable ways to interpret and present synthetic biology designs, making it easier to communicate complex genetic structures to a wide audience.

This notebook showcases the flexibility and power of SBOL3 in combination with modern visualization tools, allowing users to create detailed, accurate, and visually appealing representations of their genetic circuits.
